Create connection

In [15]:
import pandas as pd
from transformers import pipeline
import sqlite3

# Verbinde mit der SQLite-Datenbank
conn = sqlite3.connect('../swiss_travel.db') 

# SQL Query
query = """
SELECT h.hotel, h.place_cleaned, h.price_cleaned, p."Ortschaftsname", p.E, p.N
FROM hotels h
INNER JOIN (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY "Ortschaftsname" ORDER BY "Ortschaftsname") AS rn
    FROM places
) p ON h.place_cleaned = p."Ortschaftsname" AND p.rn = 1
"""

df= pd.read_sql_query(query, conn)

Add LLM functionality

In [ ]:
import requests

def ollama_query(prompt):
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "llama3",  # Or 'mistral' or other model names
            "prompt": prompt
        }
    )
    full_output = ""
    for line in response.iter_lines():
        if line:
            data = line.decode('utf-8')
            full_output += data
    return full_output

# Example prompt
result = ollama_query("Summarize this hotel information: Hotel ABC in Zurich has 4.5 stars based on 120 reviews and costs 150 CHF per night.")
print(result)